In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pres_url = "../../data/countypres_2000-2020.csv"

In [3]:
pres = pd.read_csv(pres_url)

In [4]:
geo_url = "../../data/geo.csv"

In [5]:
geo = pd.read_csv(geo_url)

In [6]:
geo.head(2)

,Sort [1],State,FIPS,County [2],County Seat(s) [3],Population(2010),Land Areakm²,Land Areami²,Water Areakm²,Water Areami²,Total Areakm²,Total Areami²,Latitude,Longitude
0,1,AL,1001,Autauga,Prattville,"54,571","1,539.582",594.436,25.776,9.952,"1,565.358",604.388,+32.536382°,–86.644490°
1,2,AL,1003,Baldwin,Bay Minette,"182,265","4,117.522","1,589.784","1,133.190",437.527,"5,250.712","2,027.311",+30.659218°,–87.746067°


In [7]:
pres.head(2)

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,AL GORE,DEMOCRAT,4942.0,17208.0,20191203,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993.0,17208.0,20191203,TOTAL


In [8]:
#pres cleanup
pres = pres.rename(columns={"county_fips":"FIPS"})
pres["FIPS"] = pres["FIPS"]
#pres = pres[(pres["state_po"]=="TX") | (pres["state_po"]=="FL") | (pres["state_po"]=="CA") | (pres["state_po"]=="IL") | (pres["state_po"]=="MI")]
pres.dropna(inplace=True)
pres_cleaned = pres

In [9]:
pres_cleaned[pres_cleaned["state_po"]=="MI"].head(2)

,year,state,state_po,county_name,FIPS,office,candidate,party,candidatevotes,totalvotes,version,mode
4800,2000,MICHIGAN,MI,ALCONA,26001.0,PRESIDENT,AL GORE,DEMOCRAT,2696.0,5997.0,20191203,TOTAL
4801,2000,MICHIGAN,MI,ALCONA,26001.0,PRESIDENT,GEORGE W. BUSH,REPUBLICAN,3152.0,5997.0,20191203,TOTAL


In [10]:
pres_cleaned.head(2)

,year,state,state_po,county_name,FIPS,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,AL GORE,DEMOCRAT,4942.0,17208.0,20191203,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993.0,17208.0,20191203,TOTAL


In [11]:
#geo cleanup
geo.replace(',','', regex=True, inplace=True)
geo["Longitude"] = geo["Longitude"].map(lambda l: float(l[1: -2])*-1 if l[0] == "–" else float(l[1: -2]))
geo["Latitude"] = geo["Latitude"].map(lambda l: float(l[1: -2])*-1 if l[0] == "–" else float(l[1: -2]))
geo["FIPS"] = geo['FIPS'].map(lambda x: int("0" + str(x)))
#geo = geo[(geo["State"]=="TX") | (geo["State"]=="FL") | (geo["State"]=="CA") | (geo["State"]=="IL")| (geo["State"]=="MI")]
geo.drop(["Land Areakm²", "Total Areakm²", "Water Areakm²", "County Seat(s) [3]", "Sort [1]", "County [2]", "State"], axis=1, inplace=True)
geo_clean = geo

In [12]:
geo_clean.head(2)

,FIPS,Population(2010),Land Areami²,Water Areami²,Total Areami²,Latitude,Longitude
0,1001,54571,594.436,9.952,604.388,32.53638,-86.64449
1,1003,182265,1589.784,437.527,2027.311,30.65921,-87.74606


In [13]:
pres_groups = list(pres_cleaned.groupby(['year', 'FIPS']).groups.keys())

In [14]:
row_cols = ["year", "FIPS", "office", "state", "dem_votes", "rep_votes", "county"]

In [15]:
pres_clean = pd.DataFrame(columns = row_cols)

In [16]:
error_group = None
for group in pres_groups:
    year = group[0]
    fips = group[1]
    office = 'president'
    g = pres_cleaned[(pres_cleaned['year'] == group[0]) & (pres_cleaned['FIPS'] == group[1])]
    g.index = np.arange(0, len(g))
    g_year = g.iloc[0]["year"]
    g_FIPS = g.iloc[0]["FIPS"]
    g_office = g.iloc[0]["office"]
    g_state = g.iloc[0]["state_po"]
    g_county = g.iloc[0]["county_name"]
    g_demvotes = g[g["party"]=="DEMOCRAT"]["candidatevotes"].values[0]
    g_repvotes = g[g["party"]=="REPUBLICAN"]["candidatevotes"].values[0]
    all_county = [[g_year, g_FIPS, g_office, g_state, g_demvotes , g_repvotes, g_county]]    
    pres_clean = pres_clean.append(pd.DataFrame(all_county, columns=row_cols))

In [17]:
#additional pres_clean cleaning:
pres_clean["total_votes"] = pres_clean["dem_votes"] + pres_clean["rep_votes"]


In [18]:
pres_clean[pres_clean["state"]=="MI"]

,year,FIPS,office,state,dem_votes,rep_votes,county,total_votes
0,2000,26001.0,PRESIDENT,MI,2696.0,3152.0,ALCONA,5848.0
0,2000,26003.0,PRESIDENT,MI,2071.0,2142.0,ALGER,4213.0
0,2000,26005.0,PRESIDENT,MI,15495.0,28197.0,ALLEGAN,43692.0
0,2000,26007.0,PRESIDENT,MI,7053.0,6769.0,ALPENA,13822.0
0,2000,26009.0,PRESIDENT,MI,4329.0,6780.0,ANTRIM,11109.0
...,...,...,...,...,...,...,...,...
0,2020,26157.0,PRESIDENT,MI,8712.0,20297.0,TUSCOLA,29009.0
0,2020,26159.0,PRESIDENT,MI,16803.0,21591.0,VAN BUREN,38394.0
0,2020,26161.0,PRESIDENT,MI,157136.0,56241.0,WASHTENAW,213377.0
0,2020,26163.0,PRESIDENT,MI,597170.0,264553.0,WAYNE,861723.0


In [19]:
# Create target column
pres_clean["target"] = pres_clean.apply(lambda row: 0 if row["dem_votes"] > row["rep_votes"] else 1, axis=1)

In [20]:
def win_percentage(row):
    if row["target"]==0:
        try:
            return round(row["dem_votes"]/row["total_votes"] *100, 2)
        except:
            return 0
    else:
        try:
            return round(row["rep_votes"]/row["total_votes"] *100, 2)
        except:
            return 0

In [21]:
def win_margin(row):
    if row["target"]==0:
        try:
            return round((row["dem_votes"]/row["total_votes"] *100) - 50, 2)
        except:
                return 0
    else:
        try:
            return round((row["rep_votes"]/row["total_votes"] *100) - 50, 2)
        except:
            return 0

In [22]:
def skew(row):
    if row["target"]==0:
        return row["win_margin"]*-1
    else:
        return row["win_margin"]

In [ ]:
def switch(row):
    if row["target"]!=:
        return row["win_margin"]*-1
    else:
        return row["win_margin"]

In [67]:
pres_clean.head(2)

,year,FIPS,office,state,dem_votes,rep_votes,county,total_votes,target,win_percentage,win_margin,skew
0,2000,1001.0,PRESIDENT,AL,4942.0,11993.0,AUTAUGA,16935.0,1,70.82,20.82,20.82
0,2000,1003.0,PRESIDENT,AL,13997.0,40872.0,BALDWIN,54869.0,1,74.49,24.49,24.49


In [94]:
df_2016 = pres_clean[pres_clean.year == 2016][['FIPS','state', 'county', 'target']].rename(columns={'target':'target_2016'})
df_2020 = pres_clean[pres_clean.year == 2020][['FIPS','state', 'county', 'target']].rename(columns={'target':'target_2020'})


In [95]:
change = df_2016.merge(df_2020, on=['state', 'county'])

In [96]:
flipped = change[(change.target_2020 - change.target_2016) != 0]

In [97]:
flipped = flipped[flipped.state.isin(['MI', 'FL', 'TX', 'IL', 'CA'])]

In [24]:
pres_clean["win_percentage"] = pres_clean.apply(win_percentage, axis=1)
pres_clean["win_margin"] = pres_clean.apply(win_margin, axis=1)
pres_clean["skew"] = pres_clean.apply(skew, axis=1)

In [ ]:
pres_clean["switch"] = pres_clean.apply

In [25]:
#for county in all unique counties, which one had the biggest change in skew

In [26]:
FIPS_groups = list(pres_cleaned.groupby(['FIPS']).groups.keys())

In [43]:
row_cols = ["all_skew_diff", "1620_skew_diff", "FIPS", "county", "state"]

In [44]:
top_skew = pd.DataFrame(columns = row_cols)

In [45]:
for fips in FIPS_groups:
    try:
        g = pres_clean[pres_clean["FIPS"]==fips]
        g.index = np.arange(0, len(g))
        g_FIPS = g.iloc[0]["FIPS"]
        g_all_skew_diff = ((np.abs(g.iloc[0]["skew"] - g.iloc[len(g)-2]["skew"]))+ (np.abs(g.iloc[1]["skew"] - g.iloc[len(g)-1]["skew"])))/2
        g_1620_skew_diff = (np.abs(g.iloc[-2]["skew"] - g.iloc[len(g)-1]["skew"]))
        g_1620_switch = g.apply(lambda )
        g_state = g.iloc[0]["state"]
        g_county = g.iloc[0]["county"]
        row = [[g_all_skew_diff, g_1620_skew_diff, g_FIPS, g_county, g_state]]
        top_skew = top_skew.append(pd.DataFrame(row, columns=row_cols))
    except:
        pass

In [32]:
j = pres_clean.merge(geo_clean, on="FIPS", how="inner")

In [33]:
j[j["year"]==2016].shape

(3114, 18)

In [60]:
o = top_skew.merge(geo_clean, on="FIPS", how="inner")

In [61]:
o.drop(["Land Areami²","Water Areami²", "Total Areami²"], axis=1, inplace=True)

In [62]:
o.sort_values(by="1620_skew_diff", ascending=False, inplace=True)

In [63]:
o.head(2)

,all_skew_diff,1620_skew_diff,FIPS,county,state,Population(2010),Latitude,Longitude
2764,22.275,43.02,49031.0,PIUTE,UT,1556,38.33588,-112.12937
2755,22.955,41.68,49013.0,DUCHESNE,UT,18607,40.28964,-110.42983


## Visualizations

#### 1 = Republican win, 0 = Democrat win

In [ ]:
sum_rep = j.rep_votes.sum()

In [ ]:
sum_dem = j.dem_votes.sum()

In [ ]:
sum_total_votes = sum_rep + sum_dem

In [ ]:
def vote_inferential(df):
    sum_rep = j.rep_votes.sum()
    sum_dem = j.dem_votes.sum()
    sum_total = sum_rep + sum_dem
    rep_vote_per = round(sum_rep/sum_total *100, 2)
    dem_vote_per = round(sum_dem/sum_total *100, 2)
    return (rep_vote_per, dem_vote_per)

In [ ]:
vote_inferential(j)

In [ ]:
j.target.value_counts(normalize=True)

In [ ]:
red = "#D33E69"
blue = "#2A3990"

In [ ]:
austin_fips = 48015

In [ ]:
#for rj[j["FIPS"]==austin_fips]

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
plt.title("Total Votes v. Total Counties Won", size=20)
plt.tick_params(axis='both', labelsize=12)
plt.ylabel("Percentage", size=15)
sns.barplot(["R. Votes", "D. Votes", "R. Counties Won", "B. Counties Won"], [46.13,57.87, 81.67,18.34], palette=[red, blue, red, blue]);

In [ ]:
wm = j[["win_margin"]]

In [ ]:
wm["skew"] = j["skew"]

In [ ]:
wm["likelihood"] = wm["win_margin"].map(lambda margin: "unlikely" if margin > 30 else "likely")

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.histplot(data=wm, x=wm["win_margin"], bins=20, ax=ax, palette = [red, blue], hue=wm["likelihood"])
plt.text(wm["win_margin"].mean()+1, 150, "Mean", color = "black", size=15, fontweight='bold')
plt.text(wm["win_margin"].mean()+1, 137, "(19%)", color = "black", size=15, fontweight='bold')
plt.vlines(wm["win_margin"].mean(), 0, 210, color="black", linewidth=6)
plt.title("Win Margin Percentage Distribution", size=20)
plt.ylabel("County Elections", size=15)
plt.xlabel("Win Margin(Percentage)", size=15)